In [10]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

from crewai.tools import tool
import yfinance as yf
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool
from langchain_openai import ChatOpenAI

# --- Tools 정의 ---
class Tools:
    @staticmethod
    @tool("One month stock price history")
    def stock_price(ticker: str) -> str:
        """
       Get one month's worth of historical stock price data for a given ticker as CSV.
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @staticmethod
    @tool("Stock news URLs")
    def stock_news(ticker: str) -> list[str]:
        """
       Fetch recent news article URLs related to the given stock ticker.
        """
        stock = yf.Ticker(ticker)
        return [article["link"] for article in stock.news]

    @staticmethod
    @tool("Company's income statement")
    def income_stmt(ticker: str) -> str:
        """
       Get the income statement of a company as CSV for the given ticker.
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @staticmethod
    @tool("Balance sheet")
    def balance_sheet(ticker: str) -> str:
        """
       Get the balance sheet of a company as CSV for the given ticker.
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @staticmethod
    @tool("Get insider transactions")
    def insider_transactions(ticker: str) -> str:
        """
       Retrieve insider trading transactions for the given stock ticker as CSV.
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()


# --- Agents 정의 ---
class Agents:
    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data.",
            verbose=True,
            tools=[Tools.stock_price],
            memory=True,
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Gathers and summarizes news and sentiment around a stock.",
            backstory="You're skilled at collecting and analyzing sentiment and news related to a stock.",
            verbose=True,
            tools=[Tools.stock_news, ScrapeWebsiteTool()],
            memory=True,
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal="Evaluates a stock's financial health using statements and insider data.",
            backstory="Experienced in financial statement analysis, you help investors understand the company’s valuation and performance.",
            verbose=True,
            tools=[
                Tools.income_stmt,
                Tools.balance_sheet,
                Tools.insider_transactions,
            ],
            memory=True,
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Make strategic investment recommendations using data from analysts.",
            backstory="A seasoned fund manager making decisions based on research, technical, and financial insights.",
            verbose=True,
            memory=True,
        )


# --- Tasks 정의 ---
class Tasks:
    def research(self, agent):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}.",
            expected_output="Detailed summary of the news and market sentiment with potential implications.",
            agent=agent,
            output_file="stock_news.md",
        )

    def technical_analysis(self, agent):
        return Task(
            description="Perform technical analysis on {company}'s stock using price movements and trends.",
            expected_output="Technical report including support/resistance, patterns, entry points, and price targets.",
            agent=agent,
            output_file="technical_analysis.md",
        )

    def financial_analysis(self, agent):
        return Task(
            description="Analyze {company}'s financial statements and insider activity to assess financial health and valuation.",
            expected_output="Comprehensive financial report including revenue, earnings, valuation, and insider trades.",
            agent=agent,
            output_file="financial_analysis.md",
        )

    def investment_recommendation(self, agent, context):
        return Task(
            description="Provide a BUY or SELL recommendation for {company}'s stock based on all prior analyses.",
            expected_output="Final investment recommendation with risk/reward analysis and rationale.",
            agent=agent,
            context=context,
            output_file="investment_recommendation.md",
        )


# --- 실행 ---
agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
technical_task = tasks.technical_analysis(technical_analyst)
financial_task = tasks.financial_analysis(financial_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_manager,
    context=[
        research_task,
        technical_task,
        financial_task,
    ],
)

crew = Crew(
    agents=[
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_manager,
    ],
    tasks=[
        research_task,
        technical_task,
        financial_task,
        recommend_task,
    ],
    verbose=True,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o"),
    memory=True,
)

company = "AMAZON"

# description 내 {company} 변수 직접 format
research_task.description = research_task.description.format(company=company)
technical_task.description = technical_task.description.format(company=company)
financial_task.description = financial_task.description.format(company=company)
recommend_task.description = recommend_task.description.format(company=company)

# kickoff
result = crew.kickoff(inputs=dict(company=company))

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9ed819a5-63f1-48a0-b814-f8c55fe66e4e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Recent Insights:                                                                                               │
│  - Thought: To provide a comprehensive and detailed summary of the latest news and market sentiment             │
│  surrounding Salesforce (CRM), I need to gather recent news articles related to the stock. This will allow me   │
│  to analyze current events, announcements, and any market reactions. Once I have the news content, I will       │
│  synthesize it with recent stock price movements and market indicators to identify notable shifts in sentiment  │
│  and potential impacts on the stock's future performance. ...                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 1713.21ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task: Gather and analyze the latest news and market sentiment surrounding the stock of AMAZON.                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

/Users/nks/Documents/crew-ai/env/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: 
Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute 
from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x

/Users/nks/Documents/crew-ai/env/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: 
Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute 
from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 42d8387a-7aa6-4baa-a9a5-18d3059c3172                                                                     │
│  Agent: Crew Manager                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9ed819a5-63f1-48a0-b814-f8c55fe66e4e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Based on a comprehensive evaluation of Amazon's current market position, financial performance,  │
│  and technical indicators, we recommend a BUY for Amazon (AMZN) stock.                                          │
│                                                                                                                 │
│  **Rationale:**                                                                                                 │
│  1. **Technical Analysis**: The stock exhibits positive technical indicators, including an ascending triangle   │
│  pattern suggesting a bullish trend with potential breakout opportunities. Key resistance levels between        │
│  226.00-232.00 have been identified, with a price target of 240.00 if these levels are decisively broken.       │
│                                                                                                                 │
│  2. **Financial Performance**: Amazon continues to demonstrate robust revenue growth, with total revenue        │
│  reaching $637,959 million in 2024 and net income at $59,248 million. This financial strength is further        │
│  supported by substantial assets and stockholder equity, reflecting a solid financial foundation.               │
│                                                                                                                 │
│  3. **Strategic Initiatives**: Amazon's investments in logistics, retail technology, and AWS continue to drive  │
│  its growth, maintaining its dominance in the e-commerce sector. The company's diverse business model           │
│  positions it well for future expansion and innovation.                                                         │
│                                                                                                                 │
│  4. **Insider Sales**: While there have been notable insider sales by key executives, these actions are likely  │
│  part of strategic financial planning and diversification rather than a lack of confidence in the company's     │
│  future performance.                                                                                            │
│                                                                                                                 │
│  5. **Market Sentiment**: Overall market sentiment towards Amazon is positive, bolstered by its growth          │
│  prospects and strategic initiatives. However, we advise keeping an eye on potential regulatory challenges and  │
│  economic factors that could affect consumer spending.                                                          │
│                                                                                                                 │
│  **Risk/Reward Analysis**: The potential rewards of investing in Amazon stock are significant, given its        │
│  strong financial position and growth trajectory. The primary risks involve regulatory scrutiny and broader     │
│  economic uncertainties. Investors should monitor these factors closely.                                        │
│                                                                                                                 │
│  In conclusion, Amazon's strong performance metrics, c

Based on a comprehensive evaluation of Amazon's current market position, financial performance, and technical indicators, we recommend a BUY for Amazon (AMZN) stock. 

**Rationale:**
1. **Technical Analysis**: The stock exhibits positive technical indicators, including an ascending triangle pattern suggesting a bullish trend with potential breakout opportunities. Key resistance levels between 226.00-232.00 have been identified, with a price target of 240.00 if these levels are decisively broken.

2. **Financial Performance**: Amazon continues to demonstrate robust revenue growth, with total revenue reaching $637,959 million in 2024 and net income at $59,248 million. This financial strength is further supported by substantial assets and stockholder equity, reflecting a solid financial foundation.

3. **Strategic Initiatives**: Amazon's investments in logistics, retail technology, and AWS continue to drive its growth, maintaining its dominance in the e-commerce sector. The company's dive